In [126]:
from skimage import io
import pandas as pd
from skimage.transform import rotate, resize
from skimage.util import pad
import random
import numpy as np

df = pd.DataFrame.from_csv("data_set.csv")

#load files to memory
img_files_paths = set(df["left"].values)
img_files = {}
for img_file_path in img_files_paths:
    img = io.imread(img_file_path)
    img_files[img_file_path] = img
    

    


In [127]:
#augment data_set by rotating positive examples
def normalize_shape(img):
    max_shape = (300,300)
    normalized = []
    if img.shape[0] < max_shape[0] and img.shape[1] < max_shape[1]:
        h, w = img.shape
        p_up = int((max_shape[0] - h)/2) +1
        p_left = int((max_shape[1] - w)/2) + 1
        normalized = pad(img, ((p_up,p_up),(p_left,p_left)), 'constant', constant_values=(0, 0))[0:max_shape[0],0:max_shape[1]]
    else:
        normalized = resize(img, max_shape, mode = 'constant')
    if normalized.shape != max_shape:
        print(normalized.shape)
    return normalized


print("data set size = {}".format(len(df)))
to_append = []
for idx, row in df.iterrows():
    if idx % 1000 == 0:
        print("sample {}/{}".format(idx, len(df)))
    df.set_value(idx, "left", normalize_shape(img_files[row["left"]]))
    df.set_value(idx, "right", normalize_shape(img_files[row["right"]]))
    if row["y"] == 1:
        for i in range(0,20):
            angle = random.randrange(360)
            rotated = rotate(img_files[row["right"]], angle, resize=True)
            rotated = normalize_shape(rotated)
            to_append.append({"left": normalize_shape(img_files[row["left"]]), "right":rotated, "y":1})
            
df = df.append(to_append, ignore_index=True)
print("data set size after augmentation = {}".format(len(df)))


data set size = 380
sample 0/380
data set size after augmentation = 780


In [128]:
max_shape = max(map(lambda x: x.shape, df["right"].values))
print(max_shape)
#for key, value in img_files.items():
 #   img_files[key] = resize(value, max_shape)

min_shape = min(map(lambda x: x.shape, df["right"].values))
print(min_shape)

(300, 300)
(300, 300)


In [135]:
from keras.layers import Input, Dense, Dropout, Conv2D, Merge, Flatten, MaxPooling2D
import keras.backend as K
from keras.models import Model
from keras import optimizers
from sklearn.model_selection import train_test_split

INPUT_SHAPE = (300,300,1)

def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


X = np.array(list(zip(df["left"].values,df["right"].values)))
Y = df["y"].values
print(X.shape)

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2)

x_l_input = Input(shape=INPUT_SHAPE)
x_r_input = Input(shape=INPUT_SHAPE)

conv_layer_1 = Conv2D(32, activation="relu", kernel_size = (5,5))
x_l = conv_layer_1(x_l_input)
x_r = conv_layer_1(x_r_input)

pool_1 = MaxPooling2D()
x_l = pool_1(x_l)
x_r = pool_1(x_r)

conv_layer_2 = Conv2D(32, activation="relu", kernel_size = (5,5))
x_l = conv_layer_2(x_l)
x_r = conv_layer_2(x_r)

pool_2 = MaxPooling2D()
x_l = pool_2(x_l)
x_r = pool_2(x_r)

dense_layer = Dense(64, activation="relu", kernel_initializer='normal')
x_l = dense_layer(x_l)
x_r = dense_layer(x_r)

x_l = Flatten()(x_l)
x_r = Flatten()(x_r)

malstm_distance = Merge(
    mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),
    output_shape=lambda x: (x[0][0], 1))([x_l, x_r])

model = Model(inputs=[x_l_input,x_r_input], outputs=malstm_distance)
model.compile(optimizer="adadelta",
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

X_train_left = np.array(list(map(lambda x: x.reshape(INPUT_SHAPE), X_train[:,0])))
X_train_right = np.array(list(map(lambda x: x.reshape(INPUT_SHAPE), X_train[:,1])))
X_val_right = np.array(list(map(lambda x: x.reshape(INPUT_SHAPE), X_validation[:,1])))
X_val_left = np.array(list(map(lambda x: x.reshape(INPUT_SHAPE), X_validation[:,0])))


trained_model = model.fit([X_train_left,X_train_right ], Y_train, batch_size=8, epochs = 10, validation_data = ([X_val_left, X_val_right], Y_validation))


(780, 2, 300, 300)


C:\Users\echomil\Anaconda3\lib\site-packages\ipykernel\__main__.py:47: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           (None, 300, 300, 1)  0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           (None, 300, 300, 1)  0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 296, 296, 32) 832         input_39[0][0]                   
                                                                 input_40[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 148, 148, 32) 0           conv2d_22[0][0]                  
          

KeyboardInterrupt: 